## Extract: Process to pull data from Source system
## Load: Process to write data to a destination system

### Common upstream & downstream systems
- **OLTP Databases: Postgres, MySQL, sqlite3, etc**
- **OLAP Databases: Snowflake, BigQuery, Clickhouse, DuckDB, etc**
- **Cloud data storage: AWS S3, GCP Cloud Store, Minio, etc**
- **Queue systems: Kafka, Redpanda, etc**
- **API**
- **Local disk: csv, excel, json, xml files**
- **SFTP\FTP server**


## Databases
When reading or writing to a database we use a database driver. Database drivers are libraries that we can use to read or write to a database.

### DuckDB
DuckDB is an in-process SQL OLAP database management system that can be used directly within a Python environment. It allows us to efficiently query large datasets without the need for a separate database server.

- Question: How do you read data from a sqlite3 database and write to a DuckDB database?
- Hint: Look at importing the database libraries for sqlite3 and duckdb and create connections to talk to the respective databases

### SQLLITE

In [4]:
import sqlite3

# Connect to an SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('2-entertainment.db')

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS movies
            (title, year, score)''')

# Insert a row of data
data = [
    ('Monty Python and the Holy Grail', 1975, 8.2),
    ('And Now for Something Completely Different', 1971, 7.5), 
    ('Sopranos', 2001, 9.9),
    ('Breaking Bad', 2008, 9.8),
    ('Vikings', 2014, 9)
]
cursor.executemany("INSERT INTO movies VALUES(?, ?, ?)", data)

# Save (commit) the changes
conn.commit()

# Close the connection
conn.close()

In [5]:
conn = sqlite3.connect('2-entertainment.db')
cur = conn.cursor()

results = cur.execute("SELECT * FROM movies")
sqllite_results = results.fetchall()
sqllite_results

[('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5),
 ('Sopranos', 2001, 9.9),
 ('Breaking Bad', 2008, 9.8),
 ('Vikings', 2014, 9)]

### DUCK DB

In [28]:
import duckdb
# Connect to an in-memory database
# con = duckdb.connect(database=":memory:")
ddb_con = duckdb.connect(database='2-ent.duckdb', read_only=False)

In [29]:
import pandas as pd

df_results = pd.DataFrame(sqllite_results, columns=['Title', 'Year', 'Score'])
df_results

,Title,Year,Score
0,Monty Python and the Holy Grail,1975,8.2
1,And Now for Something Completely Different,1971,7.5
2,Sopranos,2001,9.9
3,Breaking Bad,2008,9.8
4,Vikings,2014,9.0


In [30]:
# Query the DataFrame
df_query_result = duckdb.sql("SELECT * FROM df_results").df()
print(type(df_query_result))

<class 'pandas.core.frame.DataFrame'>


In [31]:
array_query_result = duckdb.sql("SELECT * FROM df_results").fetchall()
print(type(array_query_result))

<class 'list'>


In [32]:
# Create table
ddb_con.execute("CREATE TABLE movie_table (title VARCHAR, year INTEGER, score INTEGER)")

CatalogException: Catalog Error: Table with name "movie_table" already exists!

In [33]:
# Insert data from list
ddb_con.executemany("INSERT INTO movie_table VALUES (?, ?, ?)", sqllite_results)

In [34]:
duckdb_results = ddb_con.execute("SELECT * FROM movie_table WHERE score>9").fetchall()
ddb_con.close()

duckdb_results

[('Sopranos', 2001, 10),
 ('Breaking Bad', 2008, 10),
 ('Sopranos', 2001, 10),
 ('Breaking Bad', 2008, 10)]

- Context Manager (Best Practice):

In [ ]:
with duckdb.connect(database=':memory:') as con:

    # database operations here
    con.execute("CREATE TABLE another_table (x INTEGER)")

    # ... more code ...

# The connection is automatically closed when exit the 'with' block.

## API

- Question: How do you read data from the CoinCap API given below and write the data to a DuckDB database?
- URL: "https://api.coincap.io/v2/exchanges"
- Hint: use requests library

In [ ]:
# Fetch data from the CoinCap API

import requests
# Define the API endpoint
api_url = "https://api.coincap.io/v2/exchanges"

response = requests.get(api_url)
response.raise_for_status()  # Will raise an exception for bad status codes (4xx or 5xx)

In [41]:
response.json()

{'data': [{'exchangeId': 'binance',
   'name': 'Binance',
   'rank': '1',
   'percentTotalVolume': '35.424066953855476490000000000000000000',
   'volumeUsd': '10149714260.3777760965329186',
   'tradingPairs': '684',
   'socket': True,
   'exchangeUrl': 'https://www.binance.com/',
   'updated': 1739204520263},
  {'exchangeId': 'crypto',
   'name': 'Crypto.com Exchange',
   'rank': '2',
   'percentTotalVolume': '13.453843795067852733000000000000000000',
   'volumeUsd': '3854799348.7470854860637688',
   'tradingPairs': '114',
   'socket': False,
   'exchangeUrl': 'https://api.crypto.com/',
   'updated': 1739204491092},
  {'exchangeId': 'gdax',
   'name': 'Coinbase Pro',
   'rank': '3',
   'percentTotalVolume': '8.481626701625341722000000000000000000',
   'volumeUsd': '2430158219.7443942555927135',
   'tradingPairs': '215',
   'socket': True,
   'exchangeUrl': 'https://pro.coinbase.com/',
   'updated': 1739204533029},
  {'exchangeId': 'gate',
   'name': 'Gate',
   'rank': '4',
   'percentT

In [60]:
exchanges_df = pd.DataFrame(response.json()['data'])
exchanges_df.head(3)

,exchangeId,name,rank,percentTotalVolume,volumeUsd,tradingPairs,socket,exchangeUrl,updated
0,binance,Binance,1,35.424066953855476490000000000000000000,10149714260.3777760965329186,684,True,https://www.binance.com/,1739204520263
1,crypto,Crypto.com Exchange,2,13.453843795067852733000000000000000000,3854799348.7470854860637688,114,False,https://api.crypto.com/,1739204491092
2,gdax,Coinbase Pro,3,8.481626701625341722000000000000000000,2430158219.7443942555927135,215,True,https://pro.coinbase.com/,1739204533029


### Add Duckdb


- Connect to the DuckDB database
- Insert data into the DuckDB Exchanges table
- Prepare data for insertion
- Hint: Ensure that the data types of the data to be inserted is compatible with DuckDBs data column types in ./setup_db.py

In [62]:
# Database Connection
duck_conn = duckdb.connect(database='exchanges.duckdb')  

In [63]:
# Register the dataframe
duck_conn.register("api_data", exchanges_df) 

In [65]:
# Query the data
result = con.execute("SELECT * FROM api_data LIMIT 5").fetchdf()  
print(result)

      name  age
0    Alice   25
1      Bob   30
2  Charlie   35


## Local disk
- Question: How do you read a CSV file from local disk and write it to a database?
- Look up open function with csvreader for python

In [ ]:
###

## Web scraping
- Questions: Use beatiful soup to scrape the below website and print all the links in that website
- URL of the website to scrape
- url = 'https://example.com'


In [ ]:
###